# Load of Data

In [0]:
# Install the necessary library
%pip install prophet statsmodels seaborn matplotlib

# Import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import min, col, date_add, sum, count
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet

# Load the fact_player_activity table to a Dataframe
fact_player_activity = spark.table("dev_catalog.gold.fact_player_activity")

# Advanced Analysis of Game Player Behavior
- Player retention rate after 1, 7, and 30 days.
- Player retention rate over time.
- Average sessions per player and per game.

In [0]:
from pyspark.sql.functions import min, col, date_add, sum, count

# Retention Analysis (D1, D7, D30)
new_players = fact_player_activity.groupBy("sk_id_game").agg(min("start_time").alias("first_session"))
retention_df = fact_player_activity.join(new_players, "sk_id_game") \
    .withColumn("is_D1", (col("start_time") == date_add(col("first_session"), 1)).cast("int")) \
    .withColumn("is_D7", (col("start_time") == date_add(col("first_session"), 7)).cast("int")) \
    .withColumn("is_D30", (col("start_time") == date_add(col("first_session"), 30)).cast("int"))

retention_rates = retention_df.agg(
    (sum("is_D1") / count("PlayerID") * 100).alias("D1_Retention"),
    (sum("is_D7") / count("PlayerID") * 100).alias("D7_Retention"),
    (sum("is_D30") / count("PlayerID") * 100).alias("D30_Retention")
)
# Convert to Pandas for Visualization
retention_pd = retention_rates.toPandas()

# Plot Retention Rates with Seaborn
plt.figure(figsize=(8, 5))
sns.barplot(x=["D1", "D7", "D30"], y=retention_pd.iloc[0], palette="coolwarm")
plt.xlabel("Days")
plt.ylabel("Retention Rate (%)")
plt.title("Player Retention Over Time")
plt.ylim(0, 100)
plt.show()

# Additional Visualization: Session Duration Distribution
plt.figure(figsize=(8, 5))
sns.histplot(fact_player_activity.select("session_duration").toPandas(), bins=30, kde=True, color='purple')
plt.xlabel("Session Duration (seconds)")
plt.ylabel("Frequency")
plt.title("Distribution of Session Duration")
plt.show()

# Advanced Visualization: Retention Line Plot
plt.figure(figsize=(8, 5))
sns.lineplot(x=["D1", "D7", "D30"], y=retention_pd.iloc[0], marker="o", linestyle="-", color="blue")
plt.xlabel("Days")
plt.ylabel("Retention Rate (%)")
plt.title("Retention Trends Over Time")
plt.ylim(0, 100)
plt.grid(True)
plt.show()

# Optimization of Revenue and In-Game Purchases
- Distribution of spending by player level.
- Impact of quests and events on revenue.
- Revenue forecasting using Time Series Forecasting (ARIMA, Prophet).

In [0]:
from pyspark.sql.functions import avg, sum, col
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet

# Correlation Heatmap of Features
feature_cols = ["session_duration", "ExperiencePoints", "CurrencySpent", "QuestCompleted"]
feature_data = fact_player_activity.select(feature_cols).toPandas()
plt.figure(figsize=(8, 6))
sns.heatmap(feature_data.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()

# Distribution of Spending by Player Level
plt.figure(figsize=(8, 5))
sns.boxplot(x="Level", y="CurrencySpent", data=fact_player_activity.select("Level", "CurrencySpent").toPandas(), palette="coolwarm")
plt.xlabel("Player Level")
plt.ylabel("Currency Spent")
plt.title("Distribution of Spending by Player Level")
plt.show()

# Impact of Quests and Events on Revenue
quest_revenue_df = fact_player_activity.groupBy("QuestCompleted").agg(avg("CurrencyEarned").alias("Avg_Revenue"))
quest_revenue_pd = quest_revenue_df.toPandas()
plt.figure(figsize=(8, 5))
sns.barplot(x="QuestCompleted", y="Avg_Revenue", data=quest_revenue_pd, palette="coolwarm")
plt.xlabel("Quests Completed")
plt.ylabel("Average Revenue Earned")
plt.title("Impact of Quests on Revenue")
plt.show()

# Revenue Forecasting using ARIMA
revenue_df = fact_player_activity.groupBy("start_time").agg(sum("CurrencyEarned").alias("Total_Revenue"))
revenue_pd = revenue_df.toPandas()
revenue_pd.set_index("start_time", inplace=True)
revenue_pd.sort_index(inplace=True)

model = ARIMA(revenue_pd["Total_Revenue"], order=(5,1,0))
model_fit = model.fit()
predictions = model_fit.forecast(steps=30)

plt.figure(figsize=(10, 5))
plt.plot(revenue_pd.index, revenue_pd["Total_Revenue"], label="Actual Revenue")
plt.plot(pd.date_range(start=revenue_pd.index[-1], periods=30, freq='D'), predictions, label="Forecasted Revenue", linestyle="dashed", color="red")
plt.xlabel("Date")
plt.ylabel("Revenue")
plt.title("Revenue Forecasting (ARIMA)")
plt.legend()
plt.show()

# Revenue Forecasting using Prophet
df_prophet = revenue_pd.reset_index().rename(columns={"start_time": "ds", "Total_Revenue": "y"})
prophet = Prophet()
prophet.fit(df_prophet)
future = prophet.make_future_dataframe(periods=30)
forecast = prophet.predict(future)

plt.figure(figsize=(10, 5))
plt.plot(df_prophet.ds, df_prophet.y, label="Actual Revenue")
plt.plot(forecast.ds, forecast.yhat, label="Forecasted Revenue (Prophet)", linestyle="dashed", color="green")
plt.xlabel("Date")
plt.ylabel("Revenue")
plt.title("Revenue Forecasting (Prophet)")
plt.legend()
plt.show()

# Detection of Abnormal Behaviors
- Players with an abnormal frequency of purchases.
- Sessions with excessive playtime.
- Detection of accounts playing 24/7 without pause (sign of botting).

In [0]:
from pyspark.sql.functions import count, avg, stddev, col
import matplotlib.pyplot as plt
import seaborn as sns

# Detecting Abnormal Purchase Frequency
purchase_stats = fact_player_activity.groupBy("PlayerID").agg(count("CurrencySpent").alias("purchase_count"), avg("CurrencySpent").alias("avg_spent"), stddev("CurrencySpent").alias("std_spent"))
abnormal_purchases = purchase_stats.filter(col("purchase_count") > (col("avg_spent") + 3 * col("std_spent")))
abnormal_purchases_pd = abnormal_purchases.toPandas()

plt.figure(figsize=(8, 5))
sns.histplot(abnormal_purchases_pd["purchase_count"], bins=20, kde=True, color="red")
plt.xlabel("Number of Purchases")
plt.ylabel("Count of Players")
plt.title("Distribution of Abnormal Purchase Frequency")
plt.show()

# Detecting Sessions with Excessive Playtime
excessive_sessions = fact_player_activity.filter(col("session_duration") > (fact_player_activity.agg(avg("session_duration")).collect()[0][0] + 3 * fact_player_activity.agg(stddev("session_duration")).collect()[0][0]))
excessive_sessions_pd = excessive_sessions.toPandas()

# Convert Pandas DataFrames to Spark DataFrames
excessive_sessions_pd_spark_df = spark.createDataFrame(excessive_sessions_pd)
# Save to Delta tables
excessive_sessions_pd_spark_df.write.format("delta").mode("overwrite").saveAsTable("dev_catalog.gold.agg_excessive_sessions_playtime")

plt.figure(figsize=(8, 5))
sns.histplot(excessive_sessions_pd["session_duration"], bins=20, kde=True, color="blue")
plt.xlabel("Session Duration (seconds)")
plt.ylabel("Count of Sessions")
plt.title("Distribution of Excessive Playtime Sessions")
plt.show()

# Detecting 24/7 Botting Behavior
continuous_play = fact_player_activity.groupBy("PlayerID").agg(count("start_time").alias("session_count"))
bot_accounts = continuous_play.filter(col("session_count") > 50)  # Threshold can be adjusted
bot_accounts_pd = bot_accounts.toPandas()

plt.figure(figsize=(8, 5))
sns.histplot(bot_accounts_pd["session_count"], bins=20, kde=True, color="green")
plt.xlabel("Number of Sessions")
plt.ylabel("Count of Players")
plt.title("Distribution of Potential Bot Accounts")
plt.show()